In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
pip install python-terrier==0.10.0 nltk scikit-learn lightgbm xgboost fastrank

Note: you may need to restart the kernel to use updated packages.


In [3]:
# !pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR

In [4]:
import pandas as pd
import json
import pyterrier as pt
import random

/home/roberto/delft/inforet/moment_retrieval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#UNCOMMENT THE FOLLOWING LINE TO USE EITHER THE TVR DATASET OR THE QVH DATASET

# dataset_choice = "TVR"
dataset_choice = "QVH"

In [6]:
# Paths to JSONL files on Colab
# if dataset_choice == "TVR":
#    jsonl_train_path = '/content/drive/MyDrive/IR/text_data/tvr_train_release.jsonl'
#    jsonl_val_path = '/content/drive/MyDrive/IR/text_data/tvr_val_release.jsonl'
#    subs_path = '/content/drive/MyDrive/IR/text_data/tvqa_preprocessed_subtitles.jsonl'

# elif dataset_choice == "QVH":
#    jsonl_train_path = "/content/drive/MyDrive/IR/text_data_QVH/highlight_train_release.jsonl"
#    subs_path = "/content/drive/MyDrive/IR/text_data_QVH/subs_train.jsonl"

In [7]:
# Paths to JSONL files
if dataset_choice == "TVR":
    jsonl_train_path = 'text_data/tvr_train_release.jsonl'
    jsonl_val_path = 'text_data/tvr_val_release.jsonl'
    subs_path = 'text_data/tvqa_preprocessed_subtitles.jsonl'

elif dataset_choice == "QVH":
    jsonl_train_path = "text_data_QVH/highlight_train_release.jsonl"
    subs_path = "text_data_QVH/subs_train.jsonl"

In [8]:
# Load subtitles into a dictionary for quick access
subtitles_dict = {}
if dataset_choice == "TVR":
    with open(subs_path, 'r') as subs_file:
        for line in subs_file:
            sub_data = json.loads(line)
            subtitles_dict[sub_data['vid_name']] = sub_data['sub']
elif dataset_choice == "QVH":
    with open(subs_path, 'r') as subs_file:
        for line in subs_file:
            sub_data = json.loads(line)
            triple = sub_data['vid'].split("_")
            name = sub_data['vid']
            #turn the list name into a string
            name = "".join(name)
            if name not in subtitles_dict:
                subtitles_dict[name] = [(float(triple[-2]) + sub_data["relevant_windows"][0][0], float(triple[-2]) + sub_data["relevant_windows"][0][1], sub_data['query'])]
            else:
                subtitles_dict[name].append((float(triple[-2]) + sub_data["relevant_windows"][0][0], float(triple[-2]) + sub_data["relevant_windows"][0][1], sub_data['query']))
            
            # print(subtitles_dict[name])


In [9]:
# Function to find matching subtitles in TVR case
def find_matching_subtitles(vid_name, ts_range, subtitles_dict):
    matching_subs = []
    if vid_name in subtitles_dict:
        for subtitle in subtitles_dict[vid_name]:
            if (ts_range[0] <= subtitle['start'] <= ts_range[1]) or (ts_range[0] <= subtitle['end'] <= ts_range[1]) or (subtitle['start'] <= ts_range[0] and subtitle['end'] >= ts_range[1]):
                matching_subs.append(subtitle['text'])
    return matching_subs

In [10]:
def parse_jsonl_TVR(jsonl_path, split_type):
    # Initialize empty lists for your data
    queries_data = []
    documents_data = []
    query_rankings_data = []

    with open(jsonl_path, 'r') as file:
        for idx, line in enumerate(file):
            data = json.loads(line)
            # drop non text-based queries
            if data['type'] not in ['t']:
                continue

            # Find matching subtitles
            matching_subs = find_matching_subtitles(data['vid_name'], data['ts'], subtitles_dict)

            if matching_subs == []:
                continue

            # Extract data for the Query Set DataFrame
            queries_data.append({'qid': str(data['desc_id']), 'query': data['desc']})

            # Extract data for the Documents Set DataFrame, including matching subtitles
            documents_data.append({'docno': split_type + str(idx), 'vid_name': data['vid_name'], 'ts': data['ts'],
                                'duration': data['duration'], 'type': data['type'], 'text': "".join(matching_subs)})

            # Extract data for the Query Rankings DataFrame
            query_rankings_data.append({'qid': str(data["desc_id"]), 'query': data['desc'], 'docno': split_type + str(idx), 'rank': 1, 'score': 1.0})

    return queries_data, documents_data, query_rankings_data

def parse_jsonl_QVH(jsonl_path):
    queries_data = []
    documents_data = []
    query_rankings_data = []
    with open(jsonl_path, 'r') as file:
        for idx,line in enumerate(file):

            # Load the JSON object from the line
            data = json.loads(line)

            triple = data["vid"].split("_")
            document_name = triple[0:-2]
            document_name = data["vid"]
            start_time = float(triple[-2])
            end_time = float(triple[-1])

            if document_name not in subtitles_dict:
                #print("Document not found in subtitles: ", document_name)
                continue
            
            subs = []
            for relevant_window in data["relevant_windows"]:
                ts = [start_time+relevant_window[0], start_time+relevant_window[1]]
                subs.extend([sub for sub in subtitles_dict[document_name] if sub[0] <= ts[1] and ts[0] <= sub[1]])
                if len(subs) == 0:
                    #print("No subtitles found for ", document_name, " at time ", ts)
                    continue
            
            documents_data.append({"docno": str(idx), "vid_name": document_name, "ts": [start_time, end_time], "duration" : data["duration"], "text": " ".join([sub[2] for sub in subs])})
            queries_data.append({"qid" : str(data["qid"]), "query": data["query"]})
            query_rankings_data.append({"qid": str(data["qid"]), "query": data["query"], "docno": str(idx), "rank": 1, "score": 1.0})

    return queries_data, documents_data, query_rankings_data


In [11]:

if dataset_choice == "TVR":
    queries_data_train, documents_data_train, query_rankings_data_train = parse_jsonl_TVR(jsonl_train_path, "t")
    queries_data_val, documents_data_val, query_rankings_data_val = parse_jsonl_TVR(jsonl_val_path, "v")
    #have to create a test set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_test = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_test]
    queries_data_test = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_test]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_test]
    documents_data_test = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_test]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_test]

elif dataset_choice == "QVH":
    queries_data_train, documents_data_train, query_rankings_data_train = parse_jsonl_QVH(jsonl_train_path)
    #have to create a val set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_val = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_val]
    queries_data_val = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_val]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_val]
    documents_data_val = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_val]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_val]
    #have to create a test set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_test = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_test]
    queries_data_test = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_test]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_test]
    documents_data_test = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_test]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_test]



In [12]:
# Create DataFrames for the Query Set, Documents Set, and Query Rankings
queries_train_df = pd.DataFrame(queries_data_train)
documents_train_df = pd.DataFrame(documents_data_train)

queries_val_df = pd.DataFrame(queries_data_val)
documents_val_df = pd.DataFrame(documents_data_val)

queries_test_df = pd.DataFrame(queries_data_test)
documents_test_df = pd.DataFrame(documents_data_test)

q_rels = pd.concat([pd.DataFrame(query_rankings_data_train), pd.DataFrame(query_rankings_data_val), pd.DataFrame(query_rankings_data_test)]).reset_index(drop=True)

#print length of the dataframes
print("Train set:")
print("Queries: ", len(queries_train_df))
print("Documents: ", len(documents_train_df))

print("Val set:")
print("Queries: ", len(queries_val_df))
print("Documents: ", len(documents_val_df))

print("Test set:")
print("Queries: ", len(queries_test_df))
print("Documents: ", len(documents_test_df))

print("Query Rankings: ", len(q_rels))



Train set:
Queries:  4445
Documents:  4445
Val set:
Queries:  548
Documents:  548
Test set:
Queries:  493
Documents:  493
Query Rankings:  5486


### First Stage Retrieval [TODO: BOX]
The following part of the code will define three different first stage retrieval pipelines as an input for the trained model.

In [13]:
!ls -la /usr/lib/jvm/java-11-openjdk-amd64
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

if not pt.started():
    pt.init()

total 32
drwxr-xr-x  7 root root 4096 feb 29 19:29 .
drwxr-xr-x  4 root root 4096 feb 29 19:29 ..
drwxr-xr-x  2 root root 4096 feb 29 19:29 bin
drwxr-xr-x  4 root root 4096 feb 29 19:29 conf
lrwxrwxrwx  1 root root   42 mag  9  2023 docs -> ../../../share/doc/openjdk-11-jre-headless
drwxr-xr-x 73 root root 4096 giu  9  2023 legal
drwxr-xr-x  7 root root 4096 feb 29 19:29 lib
drwxr-xr-x  4 root root 4096 giu  9  2023 man
-rw-r--r--  1 root root 1250 gen 20 11:27 release


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [14]:
# Create an index
from pathlib import Path

indexer = pt.IterDictIndexer(
    "./index_path/",
    meta={
        "docno": 64,
        "vid_name": 64,
        "text": 131072,
    },
    stemmer="porter",
    stopwords="terrier",
    overwrite=True,
    # type=pt.index.IndexingType.MEMORY,
)

In [15]:
joint_documents_set_df = pd.concat([documents_train_df, documents_val_df, documents_test_df])

print("Length: ", len(joint_documents_set_df))

Length:  5486


In [16]:
indexed = indexer.index(
    joint_documents_set_df.to_dict(orient="records")
)

19:17:44.492 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (49) - further warnings are suppressed
19:17:46.913 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 327 empty documents


In [17]:
#These are some examples, not necessarily the best ones. Experiment with different models
num_res = 5000

# Initialize BatchRetrieve with the created index and specify BM25 as the weighting model
first_stage_bm25 = pt.BatchRetrieve(
    indexed,
    wmodel="BM25",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)


# Initialize BatchRetrieve with the created index and specify LemurTF_IDF as the weighting model
first_stage_lemurtfidf = pt.BatchRetrieve(
    indexed,
    wmodel="LemurTF_IDF",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

# Initialize BatchRetrieve with the created index and specify Hiemstra_LM as the weighting model
first_stage_hiemstra_lm = pt.BatchRetrieve(
    indexed,
    wmodel="Hiemstra_LM",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

first_stage_dlh = pt.BatchRetrieve(
    indexed,
    wmodel="DLH",
    num_results=num_res,
    metadata=['docno', 'vid_name', 'text']
)

first_stage_dfic = pt.BatchRetrieve(
    indexed,
    wmodel="DFIC",
    num_results=num_res,
    metadata=['docno', 'vid_name', 'text']
)

first_stage_lgd = pt.BatchRetrieve(
    indexed,
    wmodel="LGD",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

first_stage_in_exp_b2 = pt.BatchRetrieve(
    indexed,
    wmodel="In_expB2",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

first_stage_dirchlet = pt.BatchRetrieve(
    indexed,
    wmodel="DirichletLM",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

first_stage_coord = pt.BatchRetrieve(
    indexed,
    wmodel="CoordinateMatch",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

first_stage_coord_qe = pt.BatchRetrieve(
    indexed,
    wmodel="CoordinateMatch",
    num_results=num_res,
    controls={"qe":"on", "qemodel" : "Bo1"},
    metadata=["docno", "vid_name", "text"]
)

first_stage_pl2 = pt.BatchRetrieve(
    indexed,
    wmodel="PL2",
    num_results=num_res,
    metadata=['docno']
)

bm25_qe = pt.BatchRetrieve(indexed, wmodel="BM25", controls={"qe":"on", "qemodel" : "Bo1"}, num_results=num_res)

# Computing feature
The weighting model can be use in [pt.weighting_model.package: http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html](http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html)

In [18]:
#We create features for the second stage using the first stage retrievers

#TF-IDF based features
lemur_tf_idf_retriever = pt.BatchRetrieve(indexed, wmodel="LemurTF_IDF")
bm25_retriever = pt.BatchRetrieve(indexed, wmodel="BM25")
tf_idf_retriever = pt.BatchRetrieve(indexed, wmodel="TF_IDF")

#Language model based features
hiem_retriever = pt.BatchRetrieve(indexed, wmodel="Hiemstra_LM")
dirichlet_retriever = pt.BatchRetrieve(indexed, wmodel="DirichletLM")

#Divergence from randomness based features
pl2_retriever = pt.BatchRetrieve(indexed, wmodel="PL2")
dph_retriever = pt.BatchRetrieve(indexed, wmodel="DPH")
dlh_retriever = pt.BatchRetrieve(indexed, wmodel="DLH")

#Can add more!
coordinate_match_retriever = pt.BatchRetrieve(indexed, wmodel="CoordinateMatch")
js_kls_retrieveer = pt.BatchRetrieve(indexed, wmodel="Js_KLs")


In [19]:
# PIPELINES WITH FSR AND FEATURES

#Can test many possibilities, i just put some examples here

bm25_pipeline = ~first_stage_bm25 >> (
   tf_idf_retriever ** coordinate_match_retriever ** dirichlet_retriever ** js_kls_retrieveer ** bm25_retriever
)


lemurtf_idf_pipeline = ~first_stage_lemurtfidf >> (
    pl2_retriever ** dph_retriever ** hiem_retriever ** coordinate_match_retriever ** dirichlet_retriever ** js_kls_retrieveer
)

hiem_lm_pipeline = ~hiem_retriever >> (
    pl2_retriever ** dph_retriever ** tf_idf_retriever ** coordinate_match_retriever ** dirichlet_retriever ** js_kls_retrieveer
)

#NOTE: i believe we should not use the same first stage retriever for features AND for the first stage retrieval

In [20]:
# Prepare the queries for the pipeline, remove special characters and extra spaces
prepared_trainqueries = queries_train_df
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_train_qrels = pd.DataFrame(query_rankings_data_train)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_train_qrels['label'] = prepared_train_qrels['score']

prepared_train_qrels['label'] = prepared_train_qrels['label'].astype(int)

prepared_val_qrels = pd.DataFrame(query_rankings_data_val)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_val_qrels['label'] = prepared_val_qrels['score']

prepared_val_qrels['label'] = prepared_val_qrels['label'].astype(int)

prepared_test_qrels = pd.DataFrame(query_rankings_data_test)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_test_qrels['label'] = prepared_test_qrels['score']

prepared_test_qrels['label'] = prepared_test_qrels['label'].astype(int)

prepared_qrels = q_rels
prepared_qrels['query'] = prepared_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_qrels['query'] = prepared_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_qrels['query'] = prepared_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_qrels['label'] = prepared_qrels['score']

prepared_qrels['label'] = prepared_qrels['label'].astype(int)

prepared_valqueries = queries_val_df.reset_index()
prepared_valqueries['query'] = prepared_valqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_valqueries['query'] = prepared_valqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)

prepared_valqueries['query'] = prepared_valqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

#test set
prepared_testqueries = queries_test_df.reset_index()
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()


prepared_queries = pd.concat([prepared_trainqueries, prepared_valqueries, prepared_testqueries]).reset_index(drop=True)
prepared_qrels = pd.concat([prepared_train_qrels, prepared_val_qrels, prepared_test_qrels]).reset_index(drop=True)
prepared_documents = pd.concat([documents_train_df, documents_val_df, documents_test_df]).reset_index(drop=True)


Storage format: .csv or trec

trec(https://pyterrier.readthedocs.io/en/latest/io.html): The pt io format, but it doesn't contain feature.

In [21]:
from sklearn.svm import SVR
import xgboost as xgb
import fastrank

index = pt.IndexFactory.of("./index_path")

fsr_pipelines = [
    # {
    #    'pipe': lemurtf_idf_pipeline,
    #    'name': 'LemurTF_IDF'
    # },
    #{
    #    'pipe': bm25_pipeline,
    #    'name': 'BM25'
    #},
    #{
    #    'pipe': hiem_lm_pipeline,
    #    'name': 'Hiemstra LM'
    #}
]

learned_models = [
    # {
    #     'model': SVR(),
    #     'form': 'reg',
    #     'name': 'SVR',
    # },
    {
        'model': xgb.XGBRanker(tree_method="hist", objective="rank:ndcg"),
        'form': 'ltr',
        'name': 'XGBoost (NDCG)',
    },
    #{
    #    'model': xgb.XGBRanker(tree_method="hist", lambdarank_num_pair_per_sample=8, objective="rank:pairwise", lambdarank_pair_method="topk"),
    #    'form': 'ltr',
    #    'name': 'XGBoost (Pariwise)'
    #}
    # {
    #     'model': xgb.XGBRanker(tree_method="hist", objective="rank:map"),
    #     'form': 'ltr',
    #     'name': 'XGBoost (MAP)',
    # },
    #{
    #    'model': fastrank.TrainRequest.coordinate_ascent(),
    #    'form': 'fastrank',
    #    'name': 'FastRank Coordinate Ascent',
    #},
    #{
    #    'model': fastrank.TrainRequest.random_forest(),
    #    'form': 'fastrank',
    #    'name': 'FastRank Random Forest',
    #}
  ]

In [22]:
from pyterrier.measures import *

trained_models = [first_stage_bm25, bm25_qe, first_stage_lemurtfidf,first_stage_pl2,first_stage_dlh, first_stage_hiemstra_lm, first_stage_dirchlet,
                  first_stage_dfic,  first_stage_lgd, first_stage_in_exp_b2,
                  first_stage_coord, first_stage_coord_qe ]
names = ['BM25',  'BM25 + QE', 'LemurTF_IDF', 'PL2','DLH', 'Hiemstra LM', 'Dirichlet LM',
         'DFIC', 'LGD', 'In Exp  B2', 
         'CoordinateMatch', 'CoordinateMatch + QE',
        ]
eval_metrics = [
    MAP, NumRelRet, 
    Recall @ 1,   Recall @ 3,    Recall @ 5,    Recall @ 10, 
    Recall @ 20,  Recall @ 30,   Recall @ 50,   Recall @ 100,
    Recall @ 200, Recall @ 300,  Recall @ 500,  Recall @ 1000, 
    Recall @ 2000,Recall @ 3000, Recall @ 5000, Recall @ 10000 
]

for fsr in fsr_pipelines:
    for model in learned_models:
        names.append(f"{fsr['name']} >> {model['name']}")
        print(names[-1])
        if 'form' in model:
          pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'], form=model['form'])
        else:
          pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'])
        pipe.fit(
            prepared_trainqueries,
            prepared_train_qrels,
            prepared_valqueries,
            prepared_val_qrels
        )
        trained_models.append(pipe)

In [23]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_testqueries,
    prepared_test_qrels,
    names=names,
    eval_metrics=eval_metrics,
)

,name,AP,NumRet(rel=1),R@1,R@3,R@5,R@10,R@20,R@30,R@50,R@100,R@200,R@300,R@500,R@1000,R@2000,R@3000,R@5000,R@10000
0,BM25,0.104052,223.0,0.079108,0.105477,0.129817,0.150101,0.178499,0.208925,0.247465,0.300203,0.350913,0.369168,0.407708,0.440162,0.450304,0.452333,0.452333,0.452333
1,BM25 + QE,0.099620,347.0,0.070994,0.107505,0.119675,0.144016,0.184584,0.219067,0.247465,0.300203,0.350913,0.379310,0.417850,0.492901,0.588235,0.679513,0.703854,0.703854
2,LemurTF_IDF,0.113140,223.0,0.087221,0.119675,0.133874,0.158215,0.190669,0.215010,0.245436,0.302231,0.346856,0.377282,0.407708,0.440162,0.450304,0.452333,0.452333,0.452333
3,PL2,0.099896,223.0,0.075051,0.103448,0.123732,0.150101,0.174442,0.204868,0.247465,0.298174,0.348884,0.369168,0.401623,0.440162,0.450304,0.452333,0.452333,0.452333
4,DLH,0.098633,223.0,0.070994,0.103448,0.119675,0.154158,0.184584,0.202840,0.237323,0.300203,0.350913,0.367140,0.411765,0.440162,0.450304,0.452333,0.452333,0.452333
5,Hiemstra LM,0.105024,223.0,0.079108,0.109533,0.127789,0.164300,0.186613,0.210953,0.235294,0.296146,0.348884,0.369168,0.411765,0.440162,0.450304,0.452333,0.452333,0.452333
6,Dirichlet LM,0.091435,223.0,0.062880,0.095335,0.117647,0.152130,0.180527,0.200811,0.233266,0.269777,0.328600,0.359026,0.391481,0.442191,0.450304,0.452333,0.452333,0.452333
7,DFIC,0.094585,223.0,0.066937,0.097363,0.117647,0.146045,0.176471,0.200811,0.243408,0.302231,0.342799,0.365112,0.405680,0.442191,0.450304,0.452333,0.452333,0.452333
8,LGD,0.089713,223.0,0.060852,0.095335,0.113590,0.144016,0.172414,0.200811,0.233266,0.290061,0.350913,0.369168,0.409736,0.440162,0.450304,0.452333,0.452333,0.452333
9,In Exp B2,0.105044,223.0,0.077079,0.111562,0.135903,0.158215,0.186613,0.200811,0.257606,0.308316,0.356998,0.377282,0.409736,0.440162,0.450304,0.452333,0.452333,0.452333


In [24]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_valqueries,
    prepared_val_qrels,
    names=names,
    eval_metrics=eval_metrics,
)

,name,AP,NumRet(rel=1),R@1,R@3,R@5,R@10,R@20,R@30,R@50,R@100,R@200,R@300,R@500,R@1000,R@2000,R@3000,R@5000,R@10000
0,BM25,0.108873,251.0,0.076642,0.113139,0.133212,0.180657,0.217153,0.242701,0.277372,0.324818,0.366788,0.392336,0.423358,0.452555,0.458029,0.458029,0.458029,0.458029
1,BM25 + QE,0.108712,389.0,0.074818,0.116788,0.136861,0.175182,0.220803,0.239051,0.275547,0.326642,0.375912,0.405109,0.441606,0.481752,0.596715,0.678832,0.709854,0.709854
2,LemurTF_IDF,0.112025,251.0,0.074818,0.122263,0.144161,0.184307,0.226277,0.250000,0.277372,0.321168,0.372263,0.394161,0.423358,0.452555,0.458029,0.458029,0.458029,0.458029
3,PL2,0.106760,251.0,0.072993,0.114964,0.129562,0.173358,0.217153,0.242701,0.275547,0.312044,0.359489,0.383212,0.423358,0.452555,0.456204,0.458029,0.458029,0.458029
4,DLH,0.103463,251.0,0.071168,0.107664,0.127737,0.173358,0.209854,0.240876,0.282847,0.319343,0.359489,0.383212,0.419708,0.452555,0.458029,0.458029,0.458029,0.458029
5,Hiemstra LM,0.107611,251.0,0.071168,0.116788,0.142336,0.169708,0.224453,0.250000,0.275547,0.315693,0.355839,0.383212,0.417883,0.452555,0.458029,0.458029,0.458029,0.458029
6,Dirichlet LM,0.097680,251.0,0.067518,0.102190,0.124088,0.158759,0.195255,0.222628,0.255474,0.302920,0.354015,0.390511,0.405109,0.452555,0.458029,0.458029,0.458029,0.458029
7,DFIC,0.097472,251.0,0.067518,0.096715,0.124088,0.164234,0.202555,0.240876,0.282847,0.328467,0.361314,0.385036,0.419708,0.452555,0.458029,0.458029,0.458029,0.458029
8,LGD,0.101376,251.0,0.069343,0.107664,0.129562,0.151460,0.195255,0.233577,0.270073,0.322993,0.363139,0.397810,0.421533,0.452555,0.458029,0.458029,0.458029,0.458029
9,In Exp B2,0.114105,251.0,0.080292,0.118613,0.142336,0.177007,0.217153,0.242701,0.288321,0.332117,0.368613,0.397810,0.425182,0.452555,0.458029,0.458029,0.458029,0.458029
